# L1: Hierarchical Content Generation

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.<br>
<span style="font-size: larger;">To maintain consistency, the notebooks are run with a 'world state' consistent with the video at the start of each notebook.</span></p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

## Creating a World

In [1]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

In [2]:
world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


In [3]:
# from together import Together
from groq import Groq
from helper import get_groq_api_key

client = Groq(api_key=get_groq_api_key())

output = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ],
)

/Users/ob1/projects/aisg-agent-exploration/gaming/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
world_output =output.choices[0].message.content
print(world_output)

World Name: Kyrethia
World Description: Kyrethia is a world where cities are built on the backs of massive beasts known as the Eldrid, ancient creatures that roam the land and have been domesticated by its inhabitants, each city is a unique fusion of architecture and biology, with the Eldrid's movements and behaviors influencing the daily life of its citizens. The cities are mobile, traveling across the land in search of resources and new lands to explore. The people of Kyrethia live in harmony with the Eldrid, relying on them for transportation, protection, and sustenance. The world is full of wonder and danger as the cities navigate the challenges of life on the backs of these massive creatures.


In [5]:
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

## Generating Kingdoms

In [6]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


In [7]:
output = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
)

In [8]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

Created kingdom "Valtoria" in Kyrethia
Created kingdom "Krael" in Kyrethia
Created kingdom "Nyxoria" in Kyrethia

Kingdom 1 Description: Nyxoria is a kingdom of mystics and seers, built on the back of an Eldrid with a unique ability to sense changes in the world's energies, its leader, the Oracle Arin, is said to possess the power to communicate with the Eldrid on a deep spiritual level, the kingdom's culture is centered around the study of the arcane and the mysteries of the universe, with its people seeking to understand the workings of the world and the will of the Eldrid, Nyxoria's history is marked by its discovery of powerful magical artifacts and its role as a mediator in the conflicts between other kingdoms.


## Generating Towns

In [9]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""


In [10]:
def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\n\n'):
        town_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- {town_name} created')
        
        town_description = output.strip().split('\n')[1]\
        .split('Description: ')[1].strip()
        
        town = {
          "name": town_name,
          "description": town_description,
          "world": world['name'],
          "kingdom": kingdom['name']
        }
        towns[town_name] = town
    kingdom["towns"] = towns

In [11]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

town = list(kingdom['towns'].values())[0]
print(f'\nTown 1 Description: \
{town["description"]}')


Creating towns for kingdom: Valtoria...
- Brindlemark created
- Eldrida's Haven created
- Thornhurst created

Creating towns for kingdom: Krael...
- Blackstone created
- Ravenhurst created
- Kragnir created

Creating towns for kingdom: Nyxoria...
- Eldrador created
- Luminaria created
- Kyralia created

Town 1 Description: Eldrador is located in the eastern region of Nyxoria, situated on a raised platform on the Eldrid's back, near its left shoulder, the town is home to the famous Celestial Spire, a tower that serves as a hub for the kingdom's mystics and seers, and the Ancient Archive, a vast library containing ancient texts and scrolls, Eldrador has a rich history, having been the site of the first recorded communication between the Oracle Arin and the Eldrid.


## Generating Non-Player Characters (NPC's)

In [12]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""

In [13]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
        ],
        temperature=0 
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for output in npcs_output.split('\n\n'):
        npc_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- "{npc_name}" created')
        
        npc_description = output.strip().split('\n')[1\
        ].split('Description: ')[1].strip()
        
        npc = {
        "name": npc_name,
        "description": npc_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[npc_name] = npc
    town["npcs"] = npcs

In [14]:
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom
    break


Creating characters for the town of: Brindlemark...
- "Kaelin Darkhaven" created
- "Lyrien Flynn" created
- "Arin Vexar" created

Creating characters for the town of: Eldrida's Haven...
- "Kaelin Darkhaven" created
- "Arin the Tidesage" created
- "Lyrien Flynn" created

Creating characters for the town of: Thornhurst...
- "Kaelin Darkhaven" created
- "Elara Moonwhisper" created
- "Arin the Wanderer" created


In [15]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')


NPC 1 in Thornhurst, Valtoria:
Kaelin is a skilled warrior from Thornhurst, with a rugged appearance and a scar above his left eyebrow, he wears a suit of leather armor and wields a large sword, he has a deep sense of loyalty to the town and its people, but struggles with the trauma of his past battles against the twisted creatures, and desires to prove himself as a worthy protector of Thornhurst.


## Save the World
>Note: You will save your world state to a file different than the one shown in the video to allow future lessons to be consistent with the video. If later wish to build your own worlds, you will want to load your file rather than the saved file.

In [16]:
import json

def save_world(world, filename):
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

#save_world(world, '../shared_data/Kyropeia.json')
save_world(world, 'ogb_world.json') #save to your version